In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import os

In [2]:
# Show all columns when displaying datasets in notebook
from IPython.display import display
pd.options.display.max_columns = None

## Read and merge source files

Here we first read the source files and merge them.

In [3]:
# Areg/Wdxr/Bdsr will not be used for the following merging steps
path = "noda_cleaned"
os.listdir(path)

['Ada.csv',
 'Areg.csv',
 'Bdsr.csv',
 'Bond.csv',
 'Caze.csv',
 'Cgcd.csv',
 'Chrg.csv',
 'Dfdn.csv',
 'Disp.csv',
 'Dsum.csv',
 'Event.csv',
 'Judge.csv',
 'Parm.csv',
 'Prcn.csv',
 'Sent.csv',
 'Wdxr.csv']

In [4]:
csv_ls = os.listdir(path)
dfnames = [csv.split('.')[0] for csv in csv_ls]
df_dict = dict.fromkeys(dfnames, None)

for csv in csv_ls:
    df = pd.read_csv(os.path.join(path, csv), index_col=False)
    df_name = csv.split('.')[0]
    df_dict[df_name] = df

C:\Users\emmyp\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\emmyp\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preprocess selected tables

In [5]:
# Choose the latest bond for each charge
# AREG_SEQ_NBR was 1 for all records, meaning there were only 1 arrest for each 'sys_nbr - bofi_nbr' combination
idx = df_dict['Bond'].groupby(['SYS_NBR', 'BOFI_NBR', 'CHRG_SEQ_NBR'])['BOND_SEQ_NBR'].transform(max) == df_dict['Bond']['BOND_SEQ_NBR']
df_dict['Bond'] = df_dict['Bond'][idx]
df_dict['Bond']

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,BOND_SEQ_NBR,BOFI_NBR,SUB_CHARGE_CODE,BOND_STATUS_CODE,BOND_TYPE_CODE
0,84000020,1,1,1,1,255544,NaN,JD,RRJ
1,84000133,1,1,1,1,257683,NaN,RM,CMM
2,84000405,1,1,1,1,197485,NaN,RM,CMM
3,84000429,1,1,1,1,142503,DFM,NaN,CMM
4,84000603,1,1,1,1,258357,NaN,JD,RRJ
...,...,...,...,...,...,...,...,...,...
49662,99303723,1,1,3,1,310721,NaN,NaN,PSB
49663,99300543,1,1,2,1,388334,NaN,NaN,CMM
49664,99296442,2,1,1,1,369535,NaN,NaN,CMM
49666,99296442,1,1,2,2,361655,A04P,NaN,RRJ


In [6]:
df_dict['Chrg'].columns

Index(['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOFI_NBR',
       'CHARGE_DATE', 'LEAD_CHARGE_FLAG', 'SUB_CHARGE_CODE_CHRG', 'COUNTS',
       'ADA_CODE_CHRG', 'JUDGE_CODE_CHRG', 'BOND_MADE_AMOUNT',
       'BOND_SET_AMOUNT', 'BOND_MADE_FLAG'],
      dtype='object')

In [7]:
# Drop ADA/JUDGE from Chrg table
df_dict['Chrg'].drop(columns=['ADA_CODE_CHRG', 'JUDGE_CODE_CHRG'], inplace=True)
df_dict['Chrg']

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,BOFI_NBR,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE_CHRG,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG
0,84000020,1,1,1,255544,19840507,Y,NaN,1,2000.0,2000.0,Y
1,84000133,1,1,1,257683,19840615,Y,NaN,1,500.0,500.0,Y
2,84000154,1,1,1,255696,19840516,Y,NaN,1,0.0,0.0,N
3,84000154,1,1,2,255696,19840516,N,NaN,2,0.0,0.0,N
4,84000180,1,1,1,251021,19840524,Y,NaN,1,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...
591959,99309155,1,1,2,J117134,19991102,N,NaN,1,0.0,0.0,N
591960,99309156,1,1,1,J117135,19991102,N,NaN,1,0.0,0.0,N
591961,99309156,1,1,2,J117135,19991102,Y,NaN,1,0.0,0.0,N
591962,99309155,1,1,3,J117134,19991103,Y,NaN,1,0.0,0.0,N


In [8]:
# Drop ADA/JUDGE from Chrg table
df_dict['Disp'].drop(columns=['DISP_DATE_DISP', 'DISP_CODE_DISP'], inplace=True)
df_dict['Disp']

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,BOFI_NBR,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP
0,0,1,1,1,1,J87355,14:81,NaN,NaN,NaN
1,84000280,1,1,1,1,246712,40:967 (C) (2),ERB,NaN,NaN
2,84000405,1,1,1,1,197485,14:82,NaN,NaN,NaN
3,84000429,1,1,1,1,142503,40:966 (D) (2),DFM,NaN,NaN
4,84000873,2,1,1,1,238381,14:89,CJW,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
574564,99309156,1,1,1,1,J117135,14:69 (B),NaN,NaN,NaN
574565,99309156,1,1,2,1,J117135,14:67.10 (B),NaN,NaN,NaN
574566,99309155,1,1,1,1,J117134,14:69 (B),NaN,NaN,NaN
574567,99309155,1,1,2,1,J117134,14:67.10 (B),NaN,NaN,NaN


In [9]:
# Drop duplicated bofi_nbr from Dfdn
bofi_drop = df_dict['Dfdn'][df_dict['Dfdn'].duplicated(['BOFI_NBR'], keep='first')]['BOFI_NBR']
df_dict['Dfdn'].drop(bofi_drop.index, inplace=True)
# Drop unnecessary columns 
df_dict['Dfdn'].drop(columns=['CCN_NBR', 'ZIP_DFDN'], inplace=True)

In [10]:
# handle juvenile_flag namings
df_dict['Dfdn'].rename(columns={'JUVENILE_FLAG': 'JUVENILE_FLAG_DFDN'}, inplace=True)
df_dict['Caze'].rename(columns={'JUVENILE_FLAG': 'JUVENILE_INVOLVED_FLAG'}, inplace=True)

### Start Merging

In [11]:
#df_dict['Sent'].drop(columns=['Unnamed: 0'],inplace=True)

In [12]:
# Merge SENT and DFDN
data_merged = pd.merge(df_dict['Sent'], df_dict['Dfdn'], on='BOFI_NBR', how='inner')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK
130732,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK
130733,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50.0,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK
130734,99308953,1,1,2,1,1,J115276,14:68.4,19991115,50.0,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK


In [13]:
data_merged = pd.merge(data_merged, df_dict['Caze'], on='SYS_NBR', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN
130732,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN
130733,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50.0,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN
130734,99308953,1,1,2,1,1,J115276,14:68.4,19991115,50.0,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN


In [14]:
data_merged = pd.merge(data_merged, df_dict['Chrg'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE_CHRG,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N
130732,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N
130733,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50.0,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N
130734,99308953,1,1,2,1,1,J115276,14:68.4,19991115,50.0,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,0.0,0.0,N


In [15]:
data_merged = pd.merge(data_merged, df_dict['Cgcd'], left_on='CHARGE_CODE_SENT', right_on='CHARGE_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE_CHRG,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,40:981.3,NAR,2
130732,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4
130733,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50.0,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C),NaN,4
130734,99308953,1,1,2,1,1,J115276,14:68.4,19991115,50.0,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,0.0,0.0,N,14:68.4,NaN,3


In [16]:
data_merged = pd.merge(data_merged, df_dict['Disp'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'DISP_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE_CHRG,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,40:981.3,NAR,2,40:981.3,NaN,JEAG,NaN
130732,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN
130733,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50.0,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C),NaN,4,14:67.10 (C),NaN,JFAH,NaN
130734,99308953,1,1,2,1,1,J115276,14:68.4,19991115,50.0,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,0.0,0.0,N,14:68.4,NaN,3,14:68.4,NaN,JFAH,NaN


In [17]:
# One sentencing can have multiple conditions -> thus the number or rows increased
data_merged = pd.merge(data_merged, df_dict['Prcn'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                         'DISP_SEQ_NBR', 'SENT_SEQ_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE_CHRG,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132410,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,40:981.3,NAR,2,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132411,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132412,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50.0,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C),NaN,4,14:67.10 (C),NaN,JFAH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132413,99308953,1,1,2,1,1,J115276,14:68.4,19991115,50.0,JFAH,N,0,6,0,SIN,N,0

In [18]:
df_dict['Bond'].columns

Index(['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
       'BOND_SEQ_NBR', 'BOFI_NBR', 'SUB_CHARGE_CODE', 'BOND_STATUS_CODE',
       'BOND_TYPE_CODE'],
      dtype='object')

In [19]:
df_dict['Chrg'].columns

Index(['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOFI_NBR',
       'CHARGE_DATE', 'LEAD_CHARGE_FLAG', 'SUB_CHARGE_CODE_CHRG', 'COUNTS',
       'BOND_MADE_AMOUNT', 'BOND_SET_AMOUNT', 'BOND_MADE_FLAG'],
      dtype='object')

In [20]:
data_merged.rename(columns={'SUB_CHARGE_CODE_CHRG':'SUB_CHARGE_CODE'},inplace=True)

In [21]:
data_merged = pd.merge(data_merged, df_dict['Bond'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'BOFI_NBR', 'SUB_CHARGE_CODE'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132410,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,40:981.3,NAR,2,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132411,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132412,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50.0,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C

In [22]:
data_merged = pd.merge(data_merged, df_dict['Judge'], left_on='JUDGE_CODE_SENT', right_on='JUDGE_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132410,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,40:981.3,NAR,2,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D
132411,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D
132412,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115

In [23]:
df_dict['Dsum'].columns

Index(['SYS_NBR', 'BOFI_NBR', 'SCREEN_ADA_CODE', 'TRIAL_ADA_CODE',
       'TRIAL_DISP_CODE'],
      dtype='object')

In [24]:
df_dict['Dsum'] = df_dict['Dsum'][df_dict['Dsum']['SCREEN_ADA_CODE'].notna() | df_dict['Dsum']['TRIAL_ADA_CODE'].notna() \
                                  | df_dict['Dsum']['TRIAL_DISP_CODE'].notna()]
df_dict['Dsum'] = df_dict['Dsum'].drop_duplicates()
data_merged = pd.merge(data_merged, df_dict['Dsum'], on=['SYS_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132415,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,40:981.3,NAR,2,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D,ADAI,BHIL,50.0
132416,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50.0,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1

In [25]:
df_dict['Screen_Ada'] = df_dict['Ada']
df_dict['Screen_Ada'].columns = ['SCREEN_ADA_CODE', 'ADA_NAME_SCREEN_ADA', 'DOB_SCREEN_ADA', \
                                 'RACE_SCREEN_ADA', 'SEX_SCREEN_ADA', 'PARTY_SCREEN_ADA']
data_merged = pd.merge(data_merged, df_dict['Screen_Ada'], on='SCREEN_ADA_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132415,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,40:981.3,NAR,2,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D,ADAI,BHIL,50.0,"DAIGLE, AMIE (STASSI)",19710916.0,W,F,R
132416,99308206,1,1,5,1,1

In [26]:
df_dict['Trial_Ada'] = df_dict['Ada']
df_dict['Trial_Ada'].columns = ['TRIAL_ADA_CODE', 'ADA_NAME_TRIAL_ADA', 'DOB_TRIAL_ADA', \
                                'RACE_TRIAL_ADA', 'SEX_TRIAL_ADA', 'PARTY_TRIAL_ADA']
data_merged = pd.merge(data_merged, df_dict['Trial_Ada'], on='TRIAL_ADA_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN,"OBANNON, ROBIN CATHERINE",19630000.0,T,F,R
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN,"BELEW, CHRISTINA A H",19570409.0,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132415,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0.0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,

In [27]:
data_merged = pd.merge(data_merged, df_dict['Event'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                          'DISP_SEQ_NBR', 'SENT_SEQ_NBR', 'BOND_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged.drop(columns=['ADA_CODE_EVENT'], inplace=True)
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN,"OBANNON, ROBIN CATHERINE",19630000.0,T,F,R,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN,"BELEW, CHRISTINA A H",19570409.0,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132415,99308206,1,1,4,1,1,J116921,40:981.3,19991104,50.0,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0

In [28]:
data_merged = pd.merge(data_merged, df_dict['Parm'], left_on='EVENT_CODE_EVENT', right_on='EVENT_CODE_PARM', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT,EVENT_CODE_PARM,BOND_EVENT_FLAG,WHO_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,14:82,19911125,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN,"OBANNON, ROBIN CATHERINE",19630000.0,T,F,R,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN,"BELEW, CHRISTINA A H",19570409.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,14:89,19860602,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,15:529.1,19860730,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [29]:
for name in data_merged.columns:
    if name.endswith('_x'):
        print(name)

In [30]:
for name in data_merged.columns:
    print(name)

SYS_NBR
DFDN_SEQ_NBR
AREG_SEQ_NBR
CHRG_SEQ_NBR
DISP_SEQ_NBR
SENT_SEQ_NBR
BOFI_NBR
CHARGE_CODE_SENT
DISP_DATE_SENT
DISP_CODE_SENT
JUDGE_CODE_SENT
APPEAL_FLAG
CONFINEMENT_YEARS
CONFINEMENT_MONTHS
CONFINEMENT_DAYS
SENTENCE_TYPE
HABITUAL_OFFENDER_FLAG_SENT
ACTIVE_PROB_YEARS
ACTIVE_PROB_MONTHS
ACTIVE_PROB_DAYS
FINE_AMOUNT
VICTIM_FUND_AMOUNT
RESTITUTION_AMOUNT
INDIGENT_DEFENDER_FUND_AMOUNT
JUDICIAL_FUND_AMOUNT
FINE_SUSPENDED_FLAG
COMMUNITY_SERVICE_FLAG
CONDITION_OF_PROBATION_FLAG
CREDIT_TIME_SERVED_FLAG
SENTENCE_LOCATION
SUSPENDED_SENTENCE_FLAG
PROBATION_TYPE
PHONIC_KEY
CUSTODY_CODE
JUVENILE_FLAG_DFDN
CRIMINAL_FLAG
FBI_NBR
DOB_DFDN
HEIGHT_FEET
HEIGHT_INCHES
WEIGHT
SEX_DFDN
RACE_DFDN
SKIN_COLOR
EYE_COLOR
HAIR_COLOR
CASE_TYPE
CASE_TYPE_2
CASE_CLASS
NBR_OF_DFDN
CASE_NBR
JUVENILE_INVOLVED_FLAG
LEAD_CHARGE_CODE
CASE_CAT_1
CASE_CAT_2
CHARGE_DATE
LEAD_CHARGE_FLAG
SUB_CHARGE_CODE
COUNTS
BOND_MADE_AMOUNT
BOND_SET_AMOUNT
BOND_MADE_FLAG
CHARGE_CODE
CHARGE_CAT
CHARGE_CLASS
CHARGE_CODE_DISP
SUB_CHARGE_CO

In [31]:
temp2 = data_merged.groupby(['BOFI_NBR', 'CHARGE_CODE']).count()

In [32]:
temp2[temp2['SYS_NBR'] > 1]

,,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT,EVENT_CODE_PARM,BOND_EVENT_FLAG,WHO_CODE
BOFI_NBR,CHARGE_CODE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100565,40:967 (B) (1),2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,2,2,2,2,2,0,0,0,2,0,2,2,2,2,2,0,0,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,2,2,0,0,0,0,0,2,2,2,2,2,0,0,0,0
101271,40:967 (C) (2),3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0
101346,14:72,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,0,14,14,0,14,14,14,14,14,14,14,14,14,14,0,14,14,14,14,14,0,0,14,14,0,14,14,14,14,0,14,14,0,14,0,0,0,0,0,0,0,0,0,0,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,0,0,0,0
101365,40:966 (A) (1),3,3,3,3,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1,3,0,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,0,0,3,3,3,3,3,3,3,3,3,3,3,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,0,0,0,0
101461,14:81.2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,0,2,2,0,2,2,2,2,2,2,2,0,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y108494,14:34.3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,0,2,2,0,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,0,2,2,0,2,2,2,2,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0
Y109154,6:606,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,0,2,0,2,2,0,2,2,2,2,2,2,0,0,0,2,0,2,2,2,2,2,0,0,2,2,0,2,2,2,2,2,2,2,0,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0
Y109155,6:606,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,0,2,0,2,2,0,2,2,2,2,2,2,0,0,0,2,0,2,2,2,2,2,0,0,2,2,0,2,2,2,2,2,2,2,0,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0


In [33]:
#### PROCESSING DATE TIME ####
from datetime import datetime

In [34]:
datecols = []
for name in data_merged.columns:
    name_ = pd.Series(name.split('_'))
    if name_.isin(['DAYS', 'YEARS','MONTHS', 'HOURS','DATE', 'DOB']).any():
        datecols.append(name)
        
datecols

['DISP_DATE_SENT',
 'CONFINEMENT_YEARS',
 'CONFINEMENT_MONTHS',
 'CONFINEMENT_DAYS',
 'ACTIVE_PROB_YEARS',
 'ACTIVE_PROB_MONTHS',
 'ACTIVE_PROB_DAYS',
 'DOB_DFDN',
 'CHARGE_DATE',
 'CONDITION_YEARS',
 'CONDITION_MONTHS',
 'CONDITION_DAYS',
 'CONDITION_HOURS',
 'DOB_SCREEN_ADA',
 'DOB_TRIAL_ADA']

In [35]:
# Convert to DISP_DATE &  CHARGE_DATE to date
data_merged['DISP_DATE_SENT'] = data_merged['DISP_DATE_SENT'].apply(lambda a: datetime.strptime(str(a), '%Y%m%d').strftime('%m/%d/%Y') if a!= 0 else None)

In [36]:
temp = data_merged['CHARGE_DATE'].apply(lambda a: str(a).split('.')[0])
data_merged['CHARGE_DATE'] = [datetime.strptime(a, '%Y%m%d').strftime('%m/%d/%Y') if a!= 'nan' else None for a in temp]             

In [37]:
data_merged['CHARGE_DATE'][:5], data_merged['DISP_DATE_SENT'][:5]

(0    01/05/1985
 1    06/11/1984
 2    07/05/1983
 3    09/26/1984
 4    06/02/1986
 Name: CHARGE_DATE, dtype: object, 0    02/22/1985
 1    11/25/1991
 2    09/30/1983
 3    06/02/1986
 4    07/30/1986
 Name: DISP_DATE_SENT, dtype: object)

In [38]:
# Convert DOB to date
# Check
count = 0
ok = 0
weird = []
for a in data_merged['DOB_DFDN']:
    try:
        a = datetime.strptime(str(a), '%Y%m%d').strftime('%m/%d/%Y')
        ok+=1
    except:
        count+=1
        weird.append(a)
count, ok, np.unique(weird)

(3388,
 129032,
 array([       0, 19000000, 19450000, 19580000, 19610000, 19660000,
        19770229]))

In [39]:
# Wrong date -> Feb 28 1977
data_merged.loc[55982,'DOB_DFDN'] = 19770228
data_merged.iloc[55982]['DOB_DFDN']

19770228

In [40]:
# Assume missing month/day to be Jan 1st
temp = data_merged[data_merged['DOB_DFDN'].isin([19000000, 19450000, 19580000, 19610000, 19660000])]['DOB_DFDN']
data_merged.loc[temp.index,'DOB_DFDN'] += 101

In [41]:
# Convert to date format
data_merged['DOB_DFDN'] = data_merged['DOB_DFDN'].apply(lambda a: datetime.strptime(str(a), '%Y%m%d').strftime('%m/%d/%Y') if a!= 0 else None)

In [42]:
data_merged['DOB_DFDN']

0               None
1               None
2               None
3         08/15/1956
4         08/15/1956
             ...    
132415    08/03/1986
132416    08/03/1986
132417    12/27/1984
132418    11/11/1984
132419    08/28/1980
Name: DOB_DFDN, Length: 132420, dtype: object

In [51]:
# ADA DOB
# Check and fix 

def check_fix_date(col):
    ok = 0
    weird = []
    temp = []
    for a in col:
        try:
            a = datetime.strptime(str(a).split('.')[0], '%Y%m%d').strftime('%m/%d/%Y')
            ok+=1
            temp.append(a)
        except:
            if a % 10000 == 0:
                a+= 101
            elif a %100 == 0:
                a+= 1
            try:
                a = datetime.strptime(str(a).split('.')[0], '%Y%m%d').strftime('%m/%d/%Y')
                ok +=1
            except:
                weird.append(a)
            temp.append(a)
    print('OK: ',ok, 'Weird: ', len(weird))
    return temp, weird      

In [55]:
data_merged['DOB_SCREEN_ADA'].isna().sum()

27874

In [56]:
temp, w = check_fix_date(data_merged['DOB_SCREEN_ADA'])
print(len(temp)==len(data_merged['DOB_SCREEN_ADA']))
data_merged['DOB_SCREEN_ADA'] = temp

OK:  104546 Weird:  27874
True


In [57]:
# ADA DOB
data_merged['DOB_TRIAL_ADA'].isna().sum()

10228

In [60]:
temp, w = check_fix_date(data_merged['DOB_TRIAL_ADA'])
print(len(temp)==len(data_merged['DOB_TRIAL_ADA']))
data_merged['DOB_TRIAL_ADA'] = temp

OK:  122192 Weird:  10228
True


In [61]:
# Aggregrate time for CONFINEMENT
data_merged['CONFINEMENT_DAYS_TOTAL'] = data_merged['CONFINEMENT_YEARS']*365 + data_merged['CONFINEMENT_MONTHS']*30 + data_merged['CONFINEMENT_DAYS']
data_merged[['CONFINEMENT_YEARS', 'CONFINEMENT_MONTHS', 'CONFINEMENT_DAYS','CONFINEMENT_DAYS_TOTAL']]

,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,CONFINEMENT_DAYS_TOTAL
0,2,0,0,730
1,0,0,78,78
2,1,0,0,365
3,0,18,0,540
4,0,30,0,900
...,...,...,...,...
132415,1,0,0,365
132416,0,6,0,180
132417,0,0,7,7
132418,0,6,0,180


In [62]:
# Aggregrate time for ACTIVE_PROB
data_merged['ACTIVE_PROB_DAYS_TOTAL']= data_merged['ACTIVE_PROB_YEARS']*365 + data_merged['ACTIVE_PROB_MONTHS']*30 + data_merged['ACTIVE_PROB_DAYS']
data_merged[['ACTIVE_PROB_YEARS', 'ACTIVE_PROB_MONTHS', 'ACTIVE_PROB_DAYS','ACTIVE_PROB_DAYS_TOTAL']]

,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,ACTIVE_PROB_DAYS_TOTAL
0,1,0,0,365
1,0,0,0,0
2,2,0,0,730
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
132415,3,0,0,1095
132416,0,0,0,0
132417,0,6,0,180
132418,0,0,0,0


In [63]:
# Aggregrate time for CONDITION
data_merged['CONDITION_HOURS'] = data_merged['CONDITION_HOURS'].apply(lambda x: 0 if np.isnan(x) else round(x/24,2))
data_merged['CONDITION_YEARS'] = data_merged['CONDITION_YEARS'].apply(lambda x: 0 if np.isnan(x) else x)
data_merged['CONDITION_MONTHS'] = data_merged['CONDITION_MONTHS'].apply(lambda x: 0 if np.isnan(x) else x)
data_merged['CONDITION_DAYS'] = data_merged['CONDITION_DAYS'].apply(lambda x: 0 if np.isnan(x) else x)

data_merged['CONDITION_DAYS_TOTAL']= data_merged['CONDITION_YEARS']*365 + data_merged['CONDITION_MONTHS']*30 \
                                        + data_merged['CONDITION_DAYS']+ data_merged['CONDITION_HOURS']

data_merged[['CONDITION_YEARS', 'CONDITION_MONTHS', 'CONDITION_DAYS','CONDITION_HOURS', 'CONDITION_DAYS_TOTAL']]

,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,CONDITION_DAYS_TOTAL
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
132415,0.0,0.0,0.0,0.0,0.0
132416,0.0,0.0,0.0,0.0,0.0
132417,0.0,0.0,0.0,0.0,0.0
132418,0.0,0.0,0.0,0.0,0.0


In [64]:
data_merged['CONDITION_DAYS_TOTAL'].value_counts()

0.0      131196
10.0        119
20.0        114
365.0       105
30.0         70
          ...  
34.0          1
49.0          1
900.0         1
46.0          1
281.0         1
Name: CONDITION_DAYS_TOTAL, Length: 114, dtype: int64

In [65]:
data_merged[datecols]

,DISP_DATE_SENT,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,DOB_DFDN,CHARGE_DATE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,DOB_SCREEN_ADA,DOB_TRIAL_ADA
0,02/22/1985,2,0,0,1,0,0,None,01/05/1985,0.0,0.0,0.0,0.0,NaN,NaN
1,11/25/1991,0,0,78,0,0,0,None,06/11/1984,0.0,0.0,0.0,0.0,NaN,01/01/1963
2,09/30/1983,1,0,0,2,0,0,None,07/05/1983,0.0,0.0,0.0,0.0,NaN,04/09/1957
3,06/02/1986,0,18,0,0,0,0,08/15/1956,09/26/1984,0.0,0.0,0.0,0.0,NaN,01/01/1952
4,07/30/1986,0,30,0,0,0,0,08/15/1956,06/02/1986,0.0,0.0,0.0,0.0,NaN,01/01/1952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132415,11/04/1999,1,0,0,3,0,0,08/03/1986,10/28/1999,0.0,0.0,0.0,0.0,09/16/1971,01/01/1975
132416,11/04/1999,0,6,0,0,0,0,08/03/1986,10/28/1999,0.0,0.0,0.0,0.0,09/16/1971,01/01/1975
132417,11/15/1999,0,0,7,0,6,0,12/27/1984,05/06/1999,0.0,0.0,0.0,0.0,01/01/1972,NaN
132418,11/15/1999,0,6,0,0,0,0,11/11/1984,11/09/1999,0.0,0.0,0.0,0.0,09/16/1971,NaN


In [76]:
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT,EVENT_CODE_PARM,BOND_EVENT_FLAG,WHO_CODE,CONFINEMENT_DAYS_TOTAL,ACTIVE_PROB_DAYS_TOTAL,CONDITION_DAYS_TOTAL
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),02/22/1985,50.0,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,01/05/1985,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,730,365,0.0
1,84000405,1,1,1,1,1,197485,14:82,11/25/1991,50.0,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,06/11/1984,Y,NaN,1.0,5000.0,5000.0,Y,14:82,VIC,4,14:82,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN,"OBANNON, ROBIN CATHERINE",01/01/1963,T,F,R,NaN,NaN,NaN,NaN,78,0,0.0
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),09/30/1983,50.0,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,07/05/1983,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN,"BELEW, CHRISTINA A H",04/09/1957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,365,730,0.0
3,84000873,2,1,1,1,1,238381,14:89,06/02/1986,50.0,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,08/15/1956,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,09/26/1984,Y,CJW,1.0,100.0,100.0,Y,14:89,VIC,3,14:89,CJW,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",01/01/1952,W,M,D,NaN,NaN,NaN,NaN,540,0,0.0
4,84000873,2,1,2,1,1,238381,15:529.1,07/30/1986,30.0,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,08/15/1956,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,06/02/1986,N,CJW,1.0,0.0,0.0,N,15:529.1,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",01/01/1952,W,M,D,NaN,NaN,NaN,NaN,900,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [66]:
### Save the full version
data_merged.to_csv('data_merged_full.csv',index=False)

In [70]:
# Start dropping
drop = pd.read_csv('NODA_drop.csv')
drop = drop[drop['KEEP/DROP'].isin(['MERGED', 'DROP'])]
drop

,COLUMN NAME(KEY IDENTIFIER),KEEP/DROP,REASON TO DROP,Unnamed: 3,TOTAL COLS:,67
1,DFDN_SEQ_NBR,DROP,- Just an identifier in case we have multiple ...,NaN,NaN,NaN
2,AREG_SEQ_NBR,DROP,All entries are '1',NaN,NaN,NaN
4,DISP_SEQ_NBR,DROP,"- If we group by ['SYS_NBR', 'BOFI_NBR', 'CHRG...",NaN,NaN,NaN
9,DISP_CODE_SENT,DROP,We can drop this variable after filtering out ...,NaN,NaN,NaN
11,APPEAL_FLAG,DROP,0.14% Y and 99.86% N -> too few observations t...,NaN,NaN,NaN
13,CONFINEMENT_MONTHS,MERGED,CONFINEMENT_DAYS_TOTAL,NaN,NaN,NaN
14,CONFINEMENT_DAYS,MERGED,CONFINEMENT_DAYS_TOTAL,NaN,NaN,NaN
18,ACTIVE_PROB_MONTHS,MERGED,ACTIVE_PROB_DAYS_TOTAL,NaN,NaN,NaN
19,ACTIVE_PROB_DAYS,MERGED,ACTIVE_PROB_DAYS_TOTAL,NaN,NaN,NaN
25,FINE_SUSPENDED_FLAG,DROP,"0.22% Y, 99.78% N => too few observations to a...",NaN,NaN,NaN


In [77]:
droplist = drop['COLUMN NAME(KEY IDENTIFIER)'].tolist()
droplist+= ['CONDITION_YEARS', 'CONFINEMENT_YEARS', 'ACTIVE_PROB_YEARS']
len(droplist), droplist

(32,
 ['DFDN_SEQ_NBR',
  'AREG_SEQ_NBR',
  'DISP_SEQ_NBR',
  'DISP_CODE_SENT',
  'APPEAL_FLAG',
  'CONFINEMENT_MONTHS',
  'CONFINEMENT_DAYS',
  'ACTIVE_PROB_MONTHS',
  'ACTIVE_PROB_DAYS',
  'FINE_SUSPENDED_FLAG',
  'PHONIC_KEY',
  'FBI_NBR',
  'CASE_TYPE_2',
  'CASE_NBR',
  'CASE_CAT_2',
  'CHARGE_CODE',
  'CHARGE_CODE_DISP',
  'SUB_CHARGE_CODE_DISP',
  'JUDGE_CODE_DISP',
  'CONDITION_MONTHS',
  'CONDITION_DAYS',
  'CONDITION_HOURS',
  'BOND_SEQ_NBR',
  'JUDGE_CODE',
  'TRIAL_DISP_CODE',
  'EVENT_CODE_EVENT',
  'EVENT_CODE_PARM',
  'BOND_EVENT_FLAG',
  'WHO_CODE',
  'CONDITION_YEARS',
  'CONFINEMENT_YEARS',
  'ACTIVE_PROB_YEARS'])

In [78]:
data_merged.drop(columns=droplist, inplace=True)

In [88]:
dropmore = ['LESSER_PLEA_CHARGE_CODE_DISP', 'ADA_NAME_SCREEN_ADA', 'ADA_NAME_TRIAL_ADA']
data_merged.drop(columns=dropmore, inplace=True)

In [89]:
data_merged

,SYS_NBR,CHRG_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,JUDGE_CODE_SENT,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_CLASS,NBR_OF_DFDN,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CAT,CHARGE_CLASS,PRCN_SEQ_NBR,CONDITION_CODE,BOND_STATUS_CODE,BOND_TYPE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,CONFINEMENT_DAYS_TOTAL,ACTIVE_PROB_DAYS_TOTAL,CONDITION_DAYS_TOTAL
0,84000280,1,1,246712,40:967 (C) (2),02/22/1985,NaN,SIN,N,0.0,0.0,0.0,0.0,0.0,N,N,N,NaN,Y,S,NaN,N,N,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,3.0,1,N,40:967 (C) (2),NaN,01/05/1985,Y,ERB,1.0,0.0,0.0,N,NAR,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,730,365,0.0
1,84000405,1,1,197485,14:82,11/25/1991,NaN,SIN,N,0.0,0.0,0.0,0.0,0.0,N,N,N,NaN,N,NaN,NaN,N,N,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,N,14:82,NaN,06/11/1984,Y,NaN,1.0,5000.0,5000.0,Y,VIC,4,NaN,NaN,RM,CMM,NaN,NaN,NaN,NaN,NaN,ROBA,NaN,NaN,NaN,NaN,01/01/1963,T,F,R,78,0,0.0
2,84000429,1,1,142503,40:966 (D) (2),09/30/1983,NaN,SIN,N,0.0,0.0,0.0,0.0,0.0,N,N,N,NaN,Y,U,NaN,N,N,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,3.0,1,N,40:966 (D) (2),NaN,07/05/1983,Y,DFM,1.0,2000.0,2000.0,Y,NAR,3,NaN,NaN,NaN,CMM,NaN,NaN,NaN,NaN,NaN,CBEL,NaN,NaN,NaN,NaN,04/09/1957,NaN,NaN,NaN,365,730,0.0
3,84000873,1,1,238381,14:89,06/02/1986,NaN,SIN,Y,0.0,0.0,0.0,0.0,0.0,N,N,N,NaN,N,NaN,NaN,N,N,08/15/1956,0,0,0,F,B,NaN,NaN,NaN,VIC,3.0,2,N,14:89 (2),NaN,09/26/1984,Y,CJW,1.0,100.0,100.0,Y,VIC,3,NaN,NaN,RM,CBP,NaN,NaN,NaN,NaN,NaN,RMOS,NaN,NaN,NaN,NaN,01/01/1952,W,M,D,540,0,0.0
4,84000873,2,1,238381,15:529.1,07/30/1986,NaN,SIN,Y,0.0,0.0,0.0,0.0,0.0,N,N,N,NaN,N,NaN,NaN,N,N,08/15/1956,0,0,0,F,B,NaN,NaN,NaN,VIC,3.0,2,N,14:89 (2),NaN,06/02/1986,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,NaN,NaN,NaN,NaN,01/01/1952,W,M,D,900,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132415,99308206,4,1,J116921,40:981.3,11/04/1999,JEAG,CNR,N,30.0,0.0,0.0,0.0,0.0,N,N,N,DOC,Y,S,NaN,Y,Y,08/03/1986,5,2,102,M,N,MBR,BRO,BLK,JNP,4.0,1,N,40:966 (D) (1),NaN,10/28/1999,N,NaN,1.0,0.0,0.0,N,NAR,2,NaN,NaN,NaN,NaN,F,W,64.0,D,ADAI,BHIL,09/16/1971,W,F,R,01/01/1975,F,F,D,365,1095,0.0
132416,99308206,5,1,J116921,40:966 (D) (1),11/04/1999,JEAG,CNR,N,0.0,0.0,0.0,0.0,0.0,N,N,N,DOC,N,NaN,NaN,Y,Y,08/03/1986,5,2,102,M,N,MBR,BRO,BLK,JNP,4.0,1,N,40:966 (D) (1),NaN,10/28/1999,Y,NaN,1.0,0.0,0.0,N,NAR,4,NaN,NaN,NaN,NaN,F,W,64.0,D,ADAI,BHIL,09/16/1971,W,F,R,01/01/1975,F,F,D,180,0,0.0
132417,99306774,3,1,J96877,14:67.10 (C),11/15/1999,JFAH,SIN,N,0.0,0.0,0.0,0.0,0.0,N,N,N,DOC,N,S,NaN,Y,Y,12/27/1984,5,5,105,M,N,MBR,BRO,BLK,JGN,4.0,1,N,14:67.10 (C),NaN,05/06/1999,Y,NaN,1.0,0.0,0.0,N,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCON,LCHR,01/01/1972,W,F,R,NaN,B,F,D,7,180,0.0
132418,99308953,2,1,J115276,14:68.4,11/15/1999,JFAH,SIN,N,0.0,0.0,0.0,0.0,0.0,N,N,N,DOC,N,U,NaN,Y,Y,11/11/1984,5,2,115,F,N,DBR,BRO,BLK,JGN,3.0,1,N,14:68.4,NaN,11/09/1999,Y,NaN,1.0,0.0,0.0,N,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADAI,LCHR,09/16/1971,W,F,R,NaN,B,F,D,180,0,0.0


In [90]:
data_merged.to_csv('data_merged_dropped.csv',index=False)